In [23]:
# %pip install pyspark
# %pip install requests

In [24]:
# with open('./tmp', 'a') as f:
#     for i in range(13):
#         print(i)    
#         f.write(f"""
# # Topic {i} 

# ---
# """)

In [25]:
import requests
import io
import pandas as pd

# Функція для завантаження файлу з Google Drive і збереження його як DataFrame
def download_file_to_dataframe(file_id):
    url = f"https://drive.google.com/uc?export=download&id={file_id}"
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        content = b''
        for chunk in response.iter_content(chunk_size=32768):
            if chunk:
                content += chunk
        print(f"Файл з ID {file_id} завантажено успішно.")
        return pd.read_csv(io.BytesIO(content))
    else:
        print(f"Не вдалося завантажити файл з ID {file_id}")
        return None

# Список файлів для завантаження
files = [
    ("1GN8o8vYP8LLj3A0UWkvbgnf0YjcF3PkV", "Users.csv"),
    ("1im5cGVl1z8ejypTyNJZSQ2lqMGh6tTFN", "Products.csv"),
    ("1cDLTc0KYQfnsDWHLR0cJkF5puBpAo2Uh", "Purchases.csv")
]

# Зчитування файлів як DataFrame
dataframes = {}
for file_id, name in files:
    df = download_file_to_dataframe(file_id)
    if df is not None:
        dataframes[name] = df

# Приклад доступу до даних
for name, df in dataframes.items():
    print(f"Перші 5 рядків {name}:")
    print(df.head())


Файл з ID 1GN8o8vYP8LLj3A0UWkvbgnf0YjcF3PkV завантажено успішно.
Файл з ID 1im5cGVl1z8ejypTyNJZSQ2lqMGh6tTFN завантажено успішно.
Файл з ID 1cDLTc0KYQfnsDWHLR0cJkF5puBpAo2Uh завантажено успішно.
Перші 5 рядків Users.csv:
   user_id    name   age              email
0        1  User_1  45.0  user1@example.com
1        2  User_2  48.0  user2@example.com
2        3  User_3  36.0  user3@example.com
3        4  User_4  46.0  user4@example.com
4        5  User_5  29.0  user5@example.com
Перші 5 рядків Products.csv:
   product_id product_name     category  price
0         1.0    Product_1       Beauty    8.3
1         2.0    Product_2         Home    8.3
2         3.0    Product_3  Electronics    9.2
3         4.0    Product_4  Electronics    2.6
4         5.0    Product_5  Electronics    9.4
Перші 5 рядків Purchases.csv:
   purchase_id  user_id  product_id        date  quantity
0            1     52.0         9.0  2022-01-01       1.0
1            2     93.0        37.0  2022-01-02       8.0


In [26]:
import requests

# Функція для завантаження файлу з Google Drive
def download_file_from_google_drive(file_id, destination):
    url = f"https://drive.google.com/uc?export=download&id={file_id}"
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(destination, 'wb') as file:
            for chunk in response.iter_content(chunk_size=32768):
                if chunk:
                    file.write(chunk)
        print(f"Файл завантажено успішно: {destination}")
    else:
        print(f"Не вдалося завантажити файл з ID {file_id}")

# Список файлів для завантаження
files = [
    ("1GN8o8vYP8LLj3A0UWkvbgnf0YjcF3PkV", "./data/Users.csv"),
    ("1im5cGVl1z8ejypTyNJZSQ2lqMGh6tTFN", "./data/Products.csv"),
    ("1cDLTc0KYQfnsDWHLR0cJkF5puBpAo2Uh", "./data/Purchases.csv")
]

# Завантаження файлів
for file_id, destination in files:
    download_file_from_google_drive(file_id, destination)


Файл завантажено успішно: ./data/Users.csv
Файл завантажено успішно: ./data/Products.csv
Файл завантажено успішно: ./data/Purchases.csv


In [27]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import TimestampType, IntegerType

In [28]:
# Створюємо Spark сесію
spark = SparkSession.builder \
    .appName("PySpark Data Manipulation Homework") \
    .getOrCreate()

In [29]:
# 1. Завантажуємо датасети (10 балів)
users_df_raw = spark.read.csv('./data/Users.csv', header=True, inferSchema=True)
purchases_df_raw = spark.read.csv('./data/Purchases.csv', header=True, inferSchema=True)
products_df_raw = spark.read.csv('./data/Products.csv', header=True, inferSchema=True)


In [45]:
[df.limit(2).show() for df in (users_df_raw,
purchases_df_raw,
products_df_raw)]

+-------+------+---+-----------------+
|user_id|  name|age|            email|
+-------+------+---+-----------------+
|      1|User_1| 45|user1@example.com|
|      2|User_2| 48|user2@example.com|
+-------+------+---+-----------------+

+-----------+-------+----------+----------+--------+
|purchase_id|user_id|product_id|      date|quantity|
+-----------+-------+----------+----------+--------+
|          1|     52|         9|2022-01-01|       1|
|          2|     93|        37|2022-01-02|       8|
+-----------+-------+----------+----------+--------+

+----------+------------+--------+-----+
|product_id|product_name|category|price|
+----------+------------+--------+-----+
|         1|   Product_1|  Beauty|  8.3|
|         2|   Product_2|    Home|  8.3|
+----------+------------+--------+-----+



[None, None, None]

In [31]:
# 2. Очищаємо дані від null-значень (10 балів)
users_df = users_df_raw.dropna()
purchases_df = purchases_df_raw.dropna()
products_df = products_df_raw.dropna()

In [32]:
# 3. Загальна сума покупок за кожною категорією продуктів (20 балів)
def calc_cost_per_category(init_df, col_appendix):
    return init_df \
        .withColumn("cost", col("price") * col("quantity")) \
        .groupBy("category") \
        .agg(sum("cost").alias(f"cost_per_category_{col_appendix}"))


total_sales_by_category = calc_cost_per_category(products_df.join(purchases_df, "product_id", "inner"), "tot")
total_sales_by_category.show()


+-----------+---------------------+
|   category|cost_per_category_tot|
+-----------+---------------------+
|       Home|   1523.4999999999998|
|     Sports|   1802.4999999999998|
|Electronics|   1174.7999999999997|
|   Clothing|                790.3|
|     Beauty|    459.8999999999999|
+-----------+---------------------+



In [33]:

# 4. Загальна сума покупок за кожною категорією продуктів віком від 18 до 25 років (25 балів)
joined_filtered_df = users_df.join(purchases_df, "user_id", "inner") \
    .join(products_df, "product_id") \
    .filter((col("age") >= 18) & (col("age") <= 25))

young_users_purchases = calc_cost_per_category(joined_filtered_df, "young")
young_users_purchases.show()


+-----------+-----------------------+
|   category|cost_per_category_young|
+-----------+-----------------------+
|       Home|                  361.1|
|     Sports|     310.49999999999994|
|Electronics|                  249.6|
|   Clothing|                  245.0|
|     Beauty|     41.400000000000006|
+-----------+-----------------------+



In [44]:
young_users_purchases.show()

+-----------+-----------------------+
|   category|cost_per_category_young|
+-----------+-----------------------+
|       Home|                  361.1|
|     Sports|     310.49999999999994|
|Electronics|                  249.6|
|   Clothing|                  245.0|
|     Beauty|     41.400000000000006|
+-----------+-----------------------+



In [48]:

# 5. Частка покупок за кожною категорією товарів від сумарних витрат для вікової категорії від 18 до 25 років (25 балів)

# Перейменовуємо стовпець у total_sales_by_category
total_sales_by_category = total_sales_by_category.withColumnRenamed(
    "cost_per_category", "cost_per_category_tot"
)

# Перейменовуємо стовпець у young_users_purchases
young_users_purchases = young_users_purchases.withColumnRenamed(
    "cost_per_category", "cost_per_category_young"
)

# Виконуємо з'єднання
percentages_df = total_sales_by_category.join(
    young_users_purchases,
    on="category",
    how="inner"
).withColumn(
    "percentage",
    round(
        col("cost_per_category_young") / col("cost_per_category_tot") * 100,
        2
    )
).select(
    "category",
    "percentage"
).orderBy(
    "percentage",
    ascending=False
)



In [49]:

percentages_df.show()


+-----------+----------+
|   category|percentage|
+-----------+----------+
|   Clothing|      31.0|
|       Home|      23.7|
|Electronics|     21.25|
|     Sports|     17.23|
|     Beauty|       9.0|
+-----------+----------+



In [52]:
from pyspark.sql.functions import col, sum as spark_sum

# Крок 2: Фільтрація користувачів за віком від 18 до 25 років
users_filtered_df = users_df.filter((col('age') >= 18) & (col('age') <= 25))

# Крок 3: Об'єднання даних користувачів з покупками
purchases_users_df = purchases_df.join(users_filtered_df, on='user_id', how='inner')

# Крок 4: Об'єднання з даними про продукти
purchases_products_df = purchases_users_df.join(products_df, on='product_id', how='inner')

# Крок 5: Обчислення загальних витрат для кожної покупки
purchases_products_df = purchases_products_df.withColumn('total_purchase_price', col('price') * col('quantity'))

# Крок 6: Обчислення сумарних витрат за кожною категорією
category_expenses_df = purchases_products_df.groupBy('category').agg(
    spark_sum('total_purchase_price').alias('category_total_expense')
)

# Крок 7: Обчислення загальних витрат для всіх категорій
total_expense = purchases_products_df.agg(
    spark_sum('total_purchase_price').alias('total_expense')
).collect()[0]['total_expense']

# Крок 8: Обчислення частки витрат за кожною категорією
category_share_df = category_expenses_df.withColumn(
    'expense_share',
    round((col('category_total_expense') / total_expense) * 100, 2)
)

# Крок 9: Відображення результату
category_share_df.show()


+-----------+----------------------+-------------+
|   category|category_total_expense|expense_share|
+-----------+----------------------+-------------+
|       Home|                 361.1|         29.9|
|     Sports|    310.49999999999994|        25.71|
|Electronics|                 249.6|        20.67|
|   Clothing|                 245.0|        20.29|
|     Beauty|    41.400000000000006|         3.43|
+-----------+----------------------+-------------+



In [36]:

# 6. Топ-3 категорії продуктів з найвищим відсотком витрат споживачами віком від 18 до 25 років (10 балів)
percentages_df \
    .orderBy("percentage", ascending=False) \
    .limit(3) \
    .show()


+-----------+----------+
|   category|percentage|
+-----------+----------+
|   Clothing|      31.0|
|       Home|      23.7|
|Electronics|     21.25|
+-----------+----------+



In [17]:

# Закриваємо Spark сесію
spark.stop()

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("FirstSession").master("spark://172.233.34.56:7077").getOrCreate()


In [2]:
spark.stop()